In [ ]:
# imports
import os
from datetime import datetime
import shutil
from pathlib import Path
import urllib.request
import pandas as pd
from google.cloud import bigquery
from prefect import task, flow
from prefect_gcp.cloud_storage import GcsBucket

print("Setup Complete")


# Deployment 1
# Fetch the data from Github url
@task(log_prints=True, name="fetch-file-dataset_url")
def fetch(dataset_url: str):
    filename, _ = urllib.request.urlretrieve(dataset_url)
    return filename


# Read and tweak to fix the dtypes of pick-up and drop-off
@task(log_prints=True, name="read-and-tweak-df")
def read_tweak_df(src: str) -> pd.DataFrame:
    df = pd.read_csv(src, parse_dates=[2, 3], compression="gzip")
    return df


# Write DataFrame to a specific folder after tweaking the DataFrame
@task(log_prints=True, name="write-to-local-file")
def write_local(df: pd.DataFrame, year: int, dataset_file: str) -> Path:
    directory = Path(f"{year}")
    path_name = directory / f"{dataset_file}.parquet"
    try:
        os.makedirs(directory, exist_ok=True)
        df.to_parquet(path_name, compression="gzip", index=False)
    except OSError as error:
        print(error)
    return path_name


# https://app.prefect.cloud/account/975bd9ed-5aef-4c8a-a413-23073fef3acb/workspace/a711abba-f5ae-4b00-9315-34f92f089b77/blocks/catalog
# Upload local csv.gz file to GCS
@task(log_prints=True)
def write_gcs(path: Path) -> None:
    gcs_block = GcsBucket.load("prefect-gcs-block-ny-taxi")
    gcs_block.upload_from_path(from_path=path, to_path=path)
    print("Loaded data to GCS...Hooray!")
    return


# Delete file after uploads
@task(log_prints=True, name="deduplicate-local-data")
def deduplicate(path: Path) -> None:
    try:
        os.remove(path)
        os.rmdir(
            path,
        )
        print("Successfully deleted directory and local files...hep hep hooray")
    except OSError as error:
        print(f"The system cannot find the file specified: {error}")
    return


@flow(log_prints=True, name="etl-web-gcs")
def etl_web_gcs(year: int, month: int):
    # Parameters
    year = 2019
    month = 5
    dataset_file = f"fhv_tripdata_{year}-{month:02}"
    dataset_url = f"https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/{dataset_file}.csv.gz"

    # Execution
    # Fetch the data from Github url
    source = fetch(dataset_url)

    # no choice but to tweak the data types for us to upload this to BQ
    # Read and tweak to fix the dytpes of pick-up and drop-off
    df_tweak = read_tweak_df(source)

    # write df to local
    path_local = write_local(df_tweak, year, dataset_file)

    # Upload dataset from local to gcs
    write_gcs(path_local)

    # Delete file after uploads
    deduplicate(path_local)

    # Next step is to create a separate deployment from GCS to BigQuery


# Create Parent flow to loop
@flow(name="etl-parent-local-gcs")
def etl_parent_web_gcs(year: int, months: list):
    year = 2019
    months = [2]

    for month in months:
        etl_web_gcs(year, month)


# Run main
if __name__ == "__main__":
    etl_parent_web_gcs(year=2019, months=[2])


In [ ]:
# import
from prefect_gcp import GcpCredentials
from google.cloud import bigquery

# Deployment 2



# Upload data from GCS to BigQuery
@flow(log_prints=True, name="etl-gcs-to-bq")
def etl_gcs_to_bq(year: int, month: int):

    gcp_creds_block = GcpCredentials.load("prefect-gcs-2023-creds")
    gcp_creds = gcp_creds_block.get_credentials_from_service_account()
    client = bigquery.Client(credentials=gcp_creds)
    table_id = "dtc-de-2023.ny_taxi.ny_taxi_tripdata_2019"

    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,
        schema=[
            bigquery.SchemaField("dispatching_base_num", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("pickup_datetime", "TIMESTAMP", mode="NULLABLE"),
            bigquery.SchemaField("dropOff_datetime", "TIMESTAMP", mode="NULLABLE"),
            bigquery.SchemaField("PUlocationID", "FLOAT", mode="NULLABLE"),
            bigquery.SchemaField("DOlocationID", "FLOAT", mode="NULLABLE"),
            bigquery.SchemaField(
                "SR_Flag",
                "FLOAT",
                mode="NULLABLE",
            ),
            bigquery.SchemaField("Affiliated_base_number", "STRING", mode="NULLABLE"),
        ],
    )
    uri = f"gs://ny_taxi_bucket_de_2023/{year}/fhv_tripdata_{year}-{month:02}.csv.gz"

    load_job = client.load_table_from_uri(
        uri, table_id, job_config=job_config
    )  # Make an API request.

    load_job.result()  # Waits for the job to complete.

    destination_table = client.get_table(table_id)
    print(f"Loaded {destination_table.num_rows} rows.")


# Parent flow ETL
@flow(log_prints=True, name="etl-parent-to-bq")
def etl_parent_bq_flow(
    year: int = 2019, months: list[int] = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
):
    for month in months:
        etl_gcs_to_bq(year, month)


# run main
if __name__ == "__main__":
    year = 2019
    months = [3]

    etl_parent_bq_flow(year, months)

In [14]:
src = "/Users/reneboygarcia/Downloads/fhv_tripdata_2019-02.csv"
df = pd.read_csv(src)
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00037,2019-02-01 00:08:44,2019-02-01 00:23:35,264.0,265.0,NaN,B00037
1,B00037,2019-02-01 00:27:51,2019-02-01 00:32:54,264.0,265.0,NaN,B00037
2,B00037,2019-02-01 00:18:30,2019-02-01 00:25:45,264.0,265.0,NaN,B00037
3,B00037,2019-02-01 00:43:15,2019-02-01 00:48:29,264.0,265.0,NaN,B00037
4,B00037,2019-02-01 00:01:45,2019-02-01 00:09:13,264.0,265.0,NaN,B00037


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1707649 entries, 0 to 1707648
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   dispatching_base_num    object 
 1   pickup_datetime         object 
 2   dropOff_datetime        object 
 3   PUlocationID            float64
 4   DOlocationID            float64
 5   SR_Flag                 float64
 6   Affiliated_base_number  object 
dtypes: float64(3), object(4)
memory usage: 91.2+ MB


In [11]:
from prefect import task, flow
import requests
from prefect.filesystems import GCS
from pathlib import Path

# params
year = 2020
month =4
dataset_file = f"fhv_tripdata_{year}-{month:02}"
src_url = f"https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/{dataset_file}.csv.gz"

# path
@task()
def path_to_gcs():
    directory = Path(f"{year}")
    path_name = directory / f"{dataset_file}.csv.gz"
    return str(path_name)

# 
@flow()
def  main():
    response = requests.get(src_url)
    gcs_block = GCS.load("prefect-gcs-block")
    gcs_path = path_to_gcs()
    gcs_block.write_path(path=gcs_path, content=response.content)
    return 

# run main
if __name__ == "__main__":
    main()

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/tasks.py:261: UserWarning: A task named 'path_to_gcs' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_52847/2377279449.py:13' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/flows.py:206: UserWarning: A flow named 'main' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_52847/2377279449.py:20' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


06:50:41.441 | INFO    | prefect.engine - Created flow run 'unyielding-hippo' for flow 'main'

06:50:55.299 | INFO    | Flow run 'unyielding-hippo' - Created task run 'path_to_gcs-0e593894-0' for task 'path_to_gcs'

06:50:55.305 | INFO    | Flow run 'unyielding-hippo' - Executing 'path_to_gcs-0e593894-0' immediately...

06:50:57.469 | INFO    | Task run 'path_to_gcs-0e593894-0' - Finished in state Completed()

06:50:57.475 | ERROR   | Flow run 'unyielding-hippo' - Encountered exception during execution:
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/fsspec/registry.py", line 211, in get_filesystem_class
    register_implementation(protocol, _import_class(bit["class"]))
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/fsspec/registry.py", line 234, in _import_class
    mod = importlib.import_module(mod)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1030, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1007, in _find_and_load
  File "<frozen importlib._bootstrap>", line 984, in _find_and_load_unlocked
ModuleNotFoundError: No module named 'gcsfs'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/filesystems.py", line 401, in filesystem
    self._filesystem = fsspec.filesystem(scheme, **self.settings)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/fsspec/registry.py", line 251, in filesystem
    cls = get_filesystem_class(protocol)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/fsspec/registry.py", line 213, in get_filesystem_class
    raise ImportError(bit["err"]) from e
ImportError: Please install gcsfs to access Google Storage

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/engine.py", line 643, in orchestrate_flow_run
    result = await run_sync(flow_call)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 156, in run_sync_in_interruptible_worker_thread
    tg.start_soon(
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 662, in __aexit__
    raise exceptions[0]
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 135, in capture_worker_thread_and_result
    result = __fn(*args, **kwargs)
  File "/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_52847/2377279449.py", line 25, in main
    gcs_block.write_path(path=str(gcs_path), content=response.content)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 226, in coroutine_wrapper
    return run_async_from_worker_thread(async_fn, *args, **kwargs)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 177, in run_async_from_worker_thread
    return anyio.from_thread.run(call)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/from_thread.py", line 49, in run
    return asynclib.run_async_from_thread(func, *args)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 970, in run_async_from_thread
    return f.result()
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/concurrent/futures/_base.py", line 446, in result
    return self.__get_result()
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/c

06:50:57.879 | ERROR   | Flow run 'unyielding-hippo' - Finished in state Failed('Flow run encountered an exception. Traceback (most recent call last):\n  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/fsspec/registry.py", line 211, in get_filesystem_class\n    register_implementation(protocol, _import_class(bit["class"]))\n  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/fsspec/registry.py", line 234, in _import_class\n    mod = importlib.import_module(mod)\n  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/importlib/__init__.py", line 127, in import_module\n    return _bootstrap._gcd_import(name[level:], package, level)\n  File "<frozen importlib._bootstrap>", line 1030, in _gcd_import\n  File "<frozen importlib._bootstrap>", line 1007, in _find_and_load\n  File "<frozen importlib._bootstrap>", line 984, in _find_and_load_unlocked\nModuleNotFoundError: No module named \'gcsfs\'\n\nThe above exception was the direct cause of the following exception:\n\nTraceback (most recent call last):\n  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/filesystems.py", line 401, in filesystem\n    self._filesystem = fsspec.filesystem(scheme, **self.settings)\n  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/fsspec/registry.py", line 251, in filesystem\n    cls = get_filesystem_class(protocol)\n  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/fsspec/registry.py", line 213, in get_filesystem_class\n    raise ImportError(bit["err"]) from e\nImportError: Please install gcsfs to access Google Storage\n\nThe above exception was the direct cause of the following exception:\n\nRuntimeError: File system created with scheme \'gcs\' from base path \'gcs://prefect-de-2023/\' could not be created. You are likely missing a Python module required to use the given storage protocol.\n')

RuntimeError: File system created with scheme 'gcs' from base path 'gcs://prefect-de-2023/' could not be created. You are likely missing a Python module required to use the given storage protocol.